In [47]:
using Pkg
Pkg.DEFAULT_IO[] = stdout

IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(42) open, 0 bytes waiting)))

In [48]:
Pkg.activate("/Users/zyxu/Documents/julia/demo")

  Activating environment at `~/Documents/julia/demo/Project.toml`


In [49]:
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [324d7699] CategoricalArrays v0.10.2
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [a1dec852] Metida v0.12.4
  [ff71e718] MixedModels v4.6.0
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [50]:
using DataFrames
using LinearAlgebra
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
using Metida
using CSV
using CategoricalArrays;

In [51]:
using HighDimMixedModels

In [31]:
df = CSV.File(joinpath(dirname(pathof(Metida)), "..", "test", "csv", "1fptime.csv"); types = [String, Float64, String, String]) |> DataFrame
df[!,:response] = map(x->parse(Float64,x),df[:,:response])
df[!,:subject] = map(x->parse(Float64,x),df[:,:subject])
df[!,:time] = map(x->parse(Float64,x),df[:,:time])
transform!(df, :factor => categorical, renamecols=false)
first(df,5)

,subject,factor,time,response
,Float64,Cat…,Float64,Float64
1,1.0,0.0,1.0,20.6844
2,1.0,0.0,2.0,22.6859
3,1.0,0.0,2.5,25.5273
4,1.0,0.0,3.0,25.0032
5,1.0,0.0,6.0,30.8101


In [7]:
lmm = LMM(@formula(response ~ 0 + subject + time), df;
random = VarEffect(@covstr(1|factor), SI)
)
fit!(lmm)

Linear Mixed Model: response ~ 0 + subject + time
Random 1: 
    Model: 1|factor
    Type: SI (1), Subjects: 2
Repeated: 
    Residual only
Blocks: 2, Maximum block size: 200
Status: converged (No Errors)
    -2 logREML: 1746.4    BIC: 1758.37

    Fixed-effects parameters:
─────────────────────────────────────────────
           Coef.  Std. Error      z  Pr(>|z|)
─────────────────────────────────────────────
subject  0.01459   0.0179209   0.81    0.4156
time     1.32128   0.0202266  65.32    <1e-99
─────────────────────────────────────────────
    Variance components:
    θ vector: [23.1199, 2.06687]
  Random 1   σ²    var   534.53
  Residual   σ²    var   4.27195


In [8]:
fm = @formula(response ~ 0 + subject + time + (1|factor))
mm = fit(MixedModel, fm, df, REML=true)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /Users/zyxu/.julia/packages/ProgressMeter/Vf8un/src/ProgressMeter.jl:620
Minimizing 59 	 Time: 0:00:00 ( 7.37 ms/it)


|             |   Est. |     SE |     z |      p | σ_factor |
|:----------- | ------:| ------:| -----:| ------:| --------:|
| subject     | 0.0146 | 0.0179 |  0.81 | 0.4156 |          |
| time        | 1.3213 | 0.0202 | 65.32 | <1e-99 |          |
| (Intercept) |        |        |       |        |  23.1199 |
| Residual    | 2.0669 |        |       |        |          |


In [9]:
show(mm)

Linear mixed model fit by REML
 response ~ 0 + subject + time + (1 | factor)
 REML criterion at convergence: 1746.4048302275257

Variance components:
            Column    Variance Std.Dev. 
factor   (Intercept)  534.52916 23.11989
Residual                4.27196  2.06687
 Number of obs: 400; levels of grouping factors: 2

  Fixed-effects parameters:
─────────────────────────────────────────────
           Coef.  Std. Error      z  Pr(>|z|)
─────────────────────────────────────────────
subject  0.01459   0.0179209   0.81    0.4156
time     1.32128   0.0202266  65.32    <1e-99
─────────────────────────────────────────────

### High Dim Mixed Models

In [10]:
fm = @formula(response ~ 0 + subject + time + (1|factor))
HMM = highDimMixedModel(fm, df, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ HighDimMixedModels /Users/zyxu/.julia/dev/HighDimMixedModels/src/bricks.jl:12


In [11]:
@assert sum(HMM.M.M .!= df[:,:subject]) == 0
@assert sum(HMM.X.X .!= df[:,:time]) == 0
@assert size(HMM.Z.Z) == (400,2)

In [12]:
sigma, betaM, betaX, opt = fit(HMM, verbose = true, REML = true, alg = :LN_BOBYQA) # :LN_BOBYQA :LN_COBYLA
println("")

The initial object value is 1368.1479171012957
OPTBL: starting point [1.0, 1.0]
got 497.15843 at [471.22619, 4.27625] after 152 iterations (returned XTOL_REACHED)



In [13]:
sigma

2-element Vector{Float64}:
 471.22619093741525
   4.276254962547461

In [14]:
betaM

1-element Vector{Float64}:
 0.014629276887517986

In [15]:
betaX

1-element Vector{Float64}:
 1.3213197389570914

## High dimensional Data

In [54]:
raw = DataFrame(CSV.File("raws.csv"))
raw = raw[1:30, 1:50]

raw[!, :response] = rand(rng, 30)
raw[!, :trt] = rand(["A","B","C"], 30)
first(raw[:,45:52],5)

,...45,...46,...47,...48,...49,...50,response,trt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.0,0.0,0.0,0.0,0.0,0.0,0.585195,C
2,0.0,1.5,0.0,0.0,0.5,0.0,0.0773379,B
3,0.5,1.0,0.0,1.0,0.5,0.0,0.716628,B
4,0.5,1.0,0.0,0.0,0.5,0.0,0.320357,A
5,0.5,0.0,0.0,1.0,0.5,0.0,0.653093,C
